## Pair-Trading 전략 구현을 위한 기본 setting
- 1) os, datetime, numpy, pandas, matplotlib, matplotlib.pyplot 라이브러리 import
- 2) 엑셀 파일로 된 raw data 불러와서 dataframe 형태로 저장
- 3) 전체 기간에 na 값 있는 종목들은 drop (dropna)
- 4) 종목코드(key)와 종목명(value)을 dictionary로 저장함
- 5) 기간에 따라 train 데이터와 test 데이터로 나눔

## 1. Import Libraries

In [1]:
'''
시각화를 위해 아래 4개 install 필요함 (순서대로)
1. pyecharts / pyecharts-snapshot 패키지
 > anaconda prompt 에서 -pip install 라이브러리명 으로 설치
3. nodejs: 
 > https://nodejs.org/en/download/ 에서 다운로드
4. phantomjs 패키지
 > 설치 안되면 참고 (https://pythonspot.com/selenium-phantomjs/)
'''

import datetime as dt
import numpy as np
import pandas as pd
import pyecharts as pch
import matplotlib
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from pandas import Series
from statsmodels.tsa.stattools import adfuller

## 2. 파일 경로 / 워크북, 워크시트 로드

In [2]:
# 현재 setting 파일이 있는 폴더 경로 + '1_RawData' 폴더를 경로로 설정
# 'execute' 파일에서 rawPath 저장하기 때문에 현재 아래 코드는 안 쓴다
#### rawPath = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '1_RawData'))
#### os.path.abspath(os.path.join(os.getcwd(), '1_RawData'))

# Raw 데이터 엑셀 파일을 변수 선언
# 2015년 ~ 2018년 KOSPI 종목 일별 주가 / 수익률
wkbRaw = pd.ExcelFile(rawPath + '/All_KOSPI_Daily_20150102~.xlsx')

# train 데이터와 test 데이터 각각의 수정주가, 수익률을 df형태로 저장
# 종목코드를 header로 하고 날짜를 index로 함
allPrice = wkbRaw.parse('Comp_price',index_col=0, header =7, skiprows= range(9,14)) 
allReturn = wkbRaw.parse('Comp_return',index_col=0, header =7, skiprows= range(9,14)) 

NameError: name 'rawPath' is not defined

## 3. 데이터 가공 / 함수 선언
- 'get_data_by_date' 함수 이용해서 불러오고 싶은 기간대로 주가or수익률 데이터 불러옴
- 주가 DF : allPrice, 수익률 DF : allReturn

In [ ]:
def drop_stock_name(df : pd.DataFrame):
    if 'Name' in df.index:
        df = df.drop("Name",0)
    return df

# 아래 함수 이용해서 allPrice와 allReturn df을 날짜에 맞게 불러옴
def get_data_by_date(df : pd.DataFrame, startDate : str, endDate : str = None):
    if endDate is None:
        endDate = df.index[-1]
    df = df.loc[(df.index >= startDate) & (df.index <= endDate)]
    return df

# 해당 기간에 na값 있는 종목들 전부 제거
allPrice = allPrice.dropna(axis = 1)
allReturn = allReturn.dropna(axis = 1)

# 종목명있는 행 drop하기 전에 종목명과 종목코드 dictionary로 만듬
dicCodeName = dict(zip(allPrice.keys(),allPrice.iloc[0]))

# 종목명 있는 행 drop함. 데이터 가공 편하게 하기 위함
allPrice = drop_stock_name(allPrice)
allReturn = drop_stock_name(allReturn)

# index를 pandas의 DatetimeIndex로 변환함
allPrice.index = pd.to_datetime(allPrice.index)
allReturn.index = pd.to_datetime(allReturn.index)

#data type을 'float64'로 변경. 자연로그 취할 때나 corr 구할 때 데이터타입 걸리는 이슈 있기 때문
allPrice = allPrice.astype('float64')
allReturn = allReturn.astype('float64')

## 4. Loaded modules, 선언한 함수명, 변수명 보여주기 (미완) 

In [ ]:
# print('[imported modules list]\n: os \n  datetime as dt\n  numpy as np\n  pandas as pd\n  pyecharts as pch\n  matplotlib\n  matplotlib.pyplot as plt \n')
# print('[변수명]\n: dd')